<a href="https://colab.research.google.com/github/hydiexe/VADER-TextBlob-Analysis-Sentiment-on-DevinAI/blob/main/YT_Crawling_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-api-python-client pandas langdetect

In [ ]:
import os
import googleapiclient.discovery
import pandas as pd
from langdetect import detect

In [ ]:
api_key = 'YOUR_API_KEY'
video_id = 'YOUR_VIDEO_ID'

In [ ]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [ ]:
def get_comments(video_id, api_key, max_results=100):
    comments = []
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=max_results,
        textFormat='plainText'
    )

    response = request.execute()

    while request:
        for item in response['items']:
            published_at = item['snippet']['topLevelComment']['snippet']['publishedAt']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            if is_english(comment):
                comments.append((published_at, comment))

        if 'nextPageToken' in response:
            request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=max_results,
                pageToken=response['nextPageToken'],
                textFormat='plainText'
            )
            response = request.execute()
        else:
            break

    return comments

In [ ]:
comments = get_comments(video_id, api_key)
df = pd.DataFrame(comments, columns=['PublishedAt', 'Comment'])

In [ ]:
df.to_csv('youtube_comments.csv', index=False)